# 텍스트 유사도
### 임베딩 기법을 통해 단어의 벡터를 구한 다음 벡터 간의 거리를 계산하는 방법으로 단어 간의 유사도를 구한다.
### 본 절에서는 통계적인 방식으로 단어 간의 유사도 계산 방법에 대해 알아볼 것이다. 인공 신경망 방식의 유사도 계산법도 성능이 뛰어나지만 상황에 따라 통계적인 방식이 더 적절할 수 있으며, 개발하려는 챗봇의 주제에 따라 사용하면 챗봇 엔진 성능 향상에 도움이 될 수 있다.

## n-gram 유사도
### n-gram : 주어진 문장에서 n개의 연속적인 단어 나열을 의미하며 n개의 단어를 토큰으로 사용한다. 이는 이웃한 단어의 출현 횟수를 통계적으로 표현해 텍스트의 유사도를 계산하는 방법이다.
#### 주로 서로 다른 문장을 n-gram 방식으로 비교하여 단어의 출현 빈도에 기반한 유사도를 계산하며 논문 인용이나 도용 정도를 조사할 수 있다.

###### n에 따른 n-gram
#### 1661년 6월 뉴턴은 선생님의 제안으로 트리니티에 입학하였다.
##### n = 1: 1661년 / 6월 / 뉴턴 / 선생님 / 제안 / 트리티니 / 입학
##### n = 2 : 1661년 6월 / 6월 뉴턴 / 뉴턴 선생님 / 선생님 제안 / 제안 트리니티 / 트리니티 입학
##### .
##### .
##### .


## 2-gram 유사도 계산

In [5]:
from konlpy.tag import Komoran

In [9]:
# 어절의 단위로 2-gram 토큰을 추출하는 메소드
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams) # 추출된 토큰은 튜플 형태로  return

# 유사도 계산 메소드
def similarity(doc1, doc2):
    cnt = 0 
    for token in doc1: # doc1 과 doc2의 토큰과 몇개나 동일한지 카운트 한다.
        if token in doc2:
            cnt += 1
    return cnt/len(doc1) # 유사도 계산 공식에 대입하여 return

# 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

# 형태소 분석기에서 명사 추출
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 단어 2-gram 토큰 추출
doc1 = word_ngram(bow1, 2) # 만약 3-gram이면 (bow1, 3)이 된다.
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

# 추출된 2-gram 토큰 출력
print(doc1)
print(doc2)
print(doc3)

# 유사도 계산 및 출력
r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)


print('문장1과 2의 유사도 : ', r1)
print('문장3과 1의 유사도 : ', r2)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))
문장1과 2의 유사도 :  0.6666666666666666
문장3과 1의 유사도 :  0.0


## 코사인 유사도
### 코사인 유사도 : 두 벡터간의 코사인 각도를 이용해 유사도를 측정하는 방법이다. 이는 벡터의 크기를 고려하지 않을 때 사용한다. 예를 들면, 단어의 출현 빈도를 통해 유사도 계산을 할 때, 동일한 단어가 많이 포함되어 있을수록 벡터의 크기가 커진다. 이때, 코사인 유사도는 벡터의 크기를 고려하지 않아 결과가 안정적인 반면 n-gram의 경우 동일한 단어가 문서 내에 자주 등장하면 유사도 결과에 안 좋은 영향을 미친다.
#### 코사인은 -1~1의 값을 가지며, 방향이 완전 동일하면 1, 반대 방향이면 -1, 직각이면 0의 값을 가진다.

## 코사인 유사도 계산

In [10]:
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [26]:
# 코사인 유사도 계산 메소드
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# TDM 생성 메소드
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}
    
    for word in word_dics:
        freq_mat[word] = 0
        
    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1
    return freq_mat

# 단어 벡터 생성 메소드
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

# 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

# 형태소 분석기에서 명사 추출
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 추출한 명사를 하나로 합치고 이를 단어 묶음이라 칭함
bow = bow1 + bow2 + bow3

# 단어 묶음에서 중복을 제거해 단어 사전 구축
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)
        
# 문장별 단어 문서 행렬 계산 및 출력
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

# 문장 벡터 생성
doc1 = np.array(make_vector(freq_list1)) # 코사인 유사도 계산 메소드에 값을 넘겨줄 때, 반드시 넘파이 배열 형태로 넘겨줘야 하므로
                                         # np.array 사용함
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

# 코사인 유사도 계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)

print(r1)
print(r2)

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}
0.8333333333333335
0.20412414523193154


#### TDM : 단어 문서 행렬